In [5]:
# automatically reload dependant notebooks
%load_ext autoreload
%autoreload 2
import import_ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Single-Source Shortest Paths

Single-Source Shortest Path (SSP) algorithm is central to GPS navigation. Given a directed, weighted graph $G = (V, E)$, SSP discovers the shortest path $\delta(u, v)$ with the minimum total weight if the path $u \leadsto v$ exists, or $\infty$ if the path does not exist. See CLRS 4ed Chapter 22 *Single-Source Shortest Paths* p.604. Since the graphs used in this notebook employ weighted edges, we shall define `SSPGraph` to be `MSTGraph` as defined in [`mst.ipynb`](./mst.ipynb).

In [6]:
from graph import *
from mst import *
from util import *

SSPGraph = MSTGraph

## Bellman-Ford SSP algorithm

Bellman-Ford SSP algorithm relaxes each edge iteratively until all simple paths from the source vertex $s$ to every other vertex becomes the shortest path $\delta(u, v)$. The algorithm works with negative weights. It detects if a negative-weight cycle is reachable from the source vertex `s`, and if so, it returns a `None`, indicating that there exists no shortest path. See §22.1 *The Bellman-Ford algorithm* p.612.

We also define below the initialisation sequence `init()` as given on p.609 and the `relax()` function as given on p.610. Moreover, we factored out SSP extraction into `getSSP()`, so that we may reuse it elsewhere.

In [7]:
def init(g: SSPGraph, s: Vertex) -> None:
    # see p.609
    for u in g.getVV():
        u.par = None
        u.dis = Infinity
    s.dis = 0

def relax(e: WgtEdge) -> None:
    # see p.610
    u = e.u
    v = e.v
    if v.dis > u.dis + e.wgt:
        v.par = u
        v.dis = u.dis + e.wgt

def sspBellmanFord(g: SSPGraph, s: Vertex) -> Option[Tree]:
    # initialize
    init(g, s)
    # discover SSP in graph g
    n = g.numVV()
    # noinspection PyTypeChecker
    for i in range(1, n):  # iteration i ∈ [1, n)
        for e in g.getEE(): relax(e)  # disable type inspection, because e here is certain to be WgtEdge, not Edge
    # check for negative-weight cycle
    for e in g.getEE():
        u = e.u
        v = e.v
        if v.dis > u.dis + e.wgt: return None  # found negative-weight cycle reachable from vertex s
    return getSSP(g, s) # extract SSP p from graph g

def getSSP(g: SSPGraph, s: Vertex) -> Tree:
    p = Tree(f"{g.tag}¶")
    p.insV(s)
    for u in g.getVV():
        if u != s:
            p.insV(u)
        if not u.isRoot():
            etag = makeEtag(u.par, u)
            if g.hasE(etag):
                e = g.getE(etag)
                p.insE(e)
    return p

## Bellman-Ford DAWG

Now, we implement the `sspBellmanFordDAWG()`, which applies topological sort to the vertices to achieve a tighter time bound of $\Theta(V + E)$ when computing SSP on a directed, acyclic, weighted graph (DAWG). This algorithm is used to find critical paths in PERT charts. See §22.2 *Single-source shortest paths in directed acyclic graphs* p.616.

In [8]:
def sspBellmanFordDAWG(g: SSPGraph, s: Vertex) -> Option[Tree]:
    vv = tsort(g)
    # initialize
    init(g, s)
    # relax edges
    for u in vv:  # for each topologically sorted vertex
        for v in g.adj(u): relax(g.getE(makeEtag(u, v)))
    return getSSP(g, s)

## Dijkstra's SSP algorithm

Dijkstra's SSP algorithm follows the structure of BFS. But where as BFS assumes unitary weight edges, Dijkstra's algorithm works with non-negative edge weights. Dijkstra's algorithm uses `PriorityQueue`. When we implemented `PrimMSTGraph` in [`mst.ipynb`](./mst.ipynb), we defined `PriVertex` which can be stored in a priority queue. We shall reuse it here. But since Dijkstra's algorithm only updates the `dis` attribute, we manually copy the updated `dis` value to the `pri` attribute, so as to make the priority queue work correctly.

In [9]:
from queue import PriorityQueue

DijkstraSSPGraph = PrimMSTGraph  # uses PriVertex

def sspDijkstra(g: SSPGraph, s: PriVertex) -> Tree:
    # initialize
    init(g, s)
    s.pri = s.dis # manually update vertex priority
    b: VSet = {}  # vertex set of SSP
    q = PriorityQueue()
    for u in g.getVV(): q.put(u)
    # discover SSP in graph g
    while not q.empty():
        u: PriVertex = q.get()
        b[u.tag] = u
        for v in g.adj(u):
            if relax(g.getE(makeEtag(u, v))):
                v.pri = v.dis # manually update vertex priority
                q.queue.sort()  # rearrange q to account for decreased v.dis
    return getSSP(g, s)

# Conclusion

In this notebook, we implemented Bellman-Ford algorithm and Dijkstra's algorithm for computing single-source shortest paths. Dijkstra's algorithm is the more efficient of the two. Tests for these SSP algorithms are implemented in [`ssptest.ipynb`](./ssptest.ipynb).